In [1]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime as dt
import time

In [4]:
data1 = pd.read_csv('../raw_data/cleaned_tweets_without_dupes_120323.csv', lineterminator='\n')

In [79]:
for i in range(1,5):
    print(i)

1
2
3
4


In [12]:
data1['datetime'] = pd.to_datetime(data1['datetime'])
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347668 entries, 0 to 347667
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   Unnamed: 0     347668 non-null  int64              
 1   datetime       347668 non-null  datetime64[ns, UTC]
 2   username       347668 non-null  object             
 3   text           347668 non-null  object             
 4   date           347668 non-null  object             
 5   negative_bert  347668 non-null  float64            
 6   neutral_bert   347668 non-null  float64            
 7   positive_bert  347668 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(3)
memory usage: 21.2+ MB


In [107]:
data1['datetime'].dt.date

0        2022-09-24
1        2022-09-24
2        2022-09-24
3        2022-09-24
4        2022-09-24
            ...    
39508    2022-12-23
39509    2022-12-23
39510    2022-12-23
39511    2022-12-23
39512    2022-12-23
Name: datetime, Length: 39513, dtype: object

In [15]:
def return_tweet_volume_scraped_data(data):
    start = time.time()
    i=1
    first_time = data1['datetime'][0]
    data1['tweet_per_10_mins'] = 0
    data1['segment_tweets'] = 0
    hour = data1['datetime'].dt.hour[0]
    for index, datetime in enumerate(data1['datetime']):
        if data1['datetime'].dt.hour[index] != hour:
            if i == 150:
                prev_ind = index-1
                data1['tweet_per_10_mins'][prev_ind] = 600*i/((first_time - last_time).total_seconds())
                i = 0
                first_time = data1['datetime'][index]
            else:
                prev_ind = index-1
                data1['tweet_per_10_mins'][prev_ind] = i
                i = 0
                first_time = data1['datetime'][index]
        else: 
            pass
        data1['segment_tweets'][index] = i
        i = i + 1
        hour = data1['datetime'].dt.hour[index]
        last_time = data1['datetime'][index]

    data2 = data1.groupby(by = 'date').sum()
    data2['tweets_per_day'] = data2['tweet_per_10_mins']*48
    end = time.time()
    print(end - start)
    return data2


In [16]:
return_tweet_volume_scraped_data(data1)

/tmp/ipykernel_8377/1523555119.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['segment_tweets'][index] = i
/tmp/ipykernel_8377/1523555119.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['tweet_per_10_mins'][prev_ind] = i
/tmp/ipykernel_8377/1523555119.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['tweet_per_10_mins'][prev_ind] = 600*i/((first_time - last_time).total_seconds())


KeyboardInterrupt: 

In [ ]:
sns.barplot(data=data2, x=data2.index, y='tweets_per_day' )